# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [71]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [72]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Fortuna,US,40.5982,-124.1573,71.35,72,0,8.01,clear sky
1,1,Longyearbyen,SJ,78.2186,15.6401,40.84,93,75,2.30,broken clouds
2,2,Castro,BR,-24.7911,-50.0119,53.24,93,100,7.78,overcast clouds
3,3,Morant Bay,JM,17.8815,-76.4093,86.70,76,100,3.33,overcast clouds
4,4,Chokurdakh,RU,70.6333,147.9167,22.12,90,91,4.07,overcast clouds
5,5,Muros,ES,42.7762,-9.0603,59.09,89,5,8.43,clear sky
6,6,Faya,SA,18.3851,42.4509,66.31,24,4,5.35,clear sky
7,7,North Platte,US,41.1239,-100.7654,81.12,34,0,14.97,clear sky
8,8,Huarmey,PE,-10.0681,-78.1522,67.30,70,13,11.45,few clouds
9,9,Jamestown,US,42.0970,-79.2353,59.02,50,75,8.05,broken clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [73]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your vacation?"))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your vacation?"))

What is the minimum temperature you would like for your vacation?10
What is the maximum temperature you would like for your vacation?50


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [74]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Longyearbyen,SJ,78.2186,15.6401,40.84,93,75,2.30,broken clouds
4,4,Chokurdakh,RU,70.6333,147.9167,22.12,90,91,4.07,overcast clouds
10,10,Saskylakh,RU,71.9167,114.0833,23.76,97,100,8.01,overcast clouds
11,11,Omsukchan,RU,62.5333,155.8000,28.42,96,100,5.35,light snow
12,12,Upernavik,GL,72.7868,-56.1549,20.46,86,16,8.68,few clouds
27,27,Kaitangata,NZ,-46.2817,169.8464,37.99,82,30,6.06,scattered clouds
29,29,Tuktoyaktuk,CA,69.4541,-133.0374,32.00,93,75,18.41,broken clouds
30,30,Sergiyevsk,RU,53.9413,51.1650,45.45,94,100,10.00,overcast clouds
33,33,Verkhnevilyuysk,RU,63.4453,120.3167,24.44,96,90,4.00,overcast clouds
38,38,Mataura,NZ,-46.1927,168.8643,42.48,90,48,1.12,scattered clouds


In [75]:
preferred_cities_df.count()

City_ID                146
City                   146
Country                146
Lat                    146
Lng                    146
Max Temp               146
Humidity               146
Cloudiness             146
Wind Speed             146
Current Description    146
dtype: int64

## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [76]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna(how ='any')

# Display sample data
clean_travel_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Longyearbyen,SJ,78.2186,15.6401,40.84,93,75,2.30,broken clouds
4,4,Chokurdakh,RU,70.6333,147.9167,22.12,90,91,4.07,overcast clouds
10,10,Saskylakh,RU,71.9167,114.0833,23.76,97,100,8.01,overcast clouds
11,11,Omsukchan,RU,62.5333,155.8000,28.42,96,100,5.35,light snow
12,12,Upernavik,GL,72.7868,-56.1549,20.46,86,16,8.68,few clouds
27,27,Kaitangata,NZ,-46.2817,169.8464,37.99,82,30,6.06,scattered clouds
29,29,Tuktoyaktuk,CA,69.4541,-133.0374,32.00,93,75,18.41,broken clouds
30,30,Sergiyevsk,RU,53.9413,51.1650,45.45,94,100,10.00,overcast clouds
33,33,Verkhnevilyuysk,RU,63.4453,120.3167,24.44,96,90,4.00,overcast clouds
38,38,Mataura,NZ,-46.1927,168.8643,42.48,90,48,1.12,scattered clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [77]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy() 

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
1,Longyearbyen,SJ,40.84,broken clouds,78.2186,15.6401
4,Chokurdakh,RU,22.12,overcast clouds,70.6333,147.9167
10,Saskylakh,RU,23.76,overcast clouds,71.9167,114.0833
11,Omsukchan,RU,28.42,light snow,62.5333,155.8000
12,Upernavik,GL,20.46,few clouds,72.7868,-56.1549
27,Kaitangata,NZ,37.99,scattered clouds,-46.2817,169.8464
29,Tuktoyaktuk,CA,32.00,broken clouds,69.4541,-133.0374
30,Sergiyevsk,RU,45.45,overcast clouds,53.9413,51.1650
33,Verkhnevilyuysk,RU,24.44,overcast clouds,63.4453,120.3167
38,Mataura,NZ,42.48,scattered clouds,-46.1927,168.8643


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [78]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Longyearbyen,SJ,40.84,broken clouds,78.2186,15.6401,
4,Chokurdakh,RU,22.12,overcast clouds,70.6333,147.9167,
10,Saskylakh,RU,23.76,overcast clouds,71.9167,114.0833,
11,Omsukchan,RU,28.42,light snow,62.5333,155.8000,
12,Upernavik,GL,20.46,few clouds,72.7868,-56.1549,
27,Kaitangata,NZ,37.99,scattered clouds,-46.2817,169.8464,
29,Tuktoyaktuk,CA,32.00,broken clouds,69.4541,-133.0374,
30,Sergiyevsk,RU,45.45,overcast clouds,53.9413,51.1650,
33,Verkhnevilyuysk,RU,24.44,overcast clouds,63.4453,120.3167,
38,Mataura,NZ,42.48,scattered clouds,-46.1927,168.8643,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [79]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [80]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [81]:
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Longyearbyen,SJ,40.84,broken clouds,78.2186,15.6401,"Radisson Blu Polar Hotel, Spitsbergen"
4,Chokurdakh,RU,22.12,overcast clouds,70.6333,147.9167,
10,Saskylakh,RU,23.76,overcast clouds,71.9167,114.0833,
11,Omsukchan,RU,28.42,light snow,62.5333,155.8000,
12,Upernavik,GL,20.46,few clouds,72.7868,-56.1549,Café de Upernavik
27,Kaitangata,NZ,37.99,scattered clouds,-46.2817,169.8464,Kaitangata Motor Camp
29,Tuktoyaktuk,CA,32.00,broken clouds,69.4541,-133.0374,End of The Road Ltd.
30,Sergiyevsk,RU,45.45,overcast clouds,53.9413,51.1650,Vertikal'
33,Verkhnevilyuysk,RU,24.44,overcast clouds,63.4453,120.3167,Gostinitsa Mila
38,Mataura,NZ,42.48,scattered clouds,-46.1927,168.8643,Ellie's Villa


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [82]:
# Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna(how ='any')

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Longyearbyen,SJ,40.84,broken clouds,78.2186,15.6401,"Radisson Blu Polar Hotel, Spitsbergen"
12,Upernavik,GL,20.46,few clouds,72.7868,-56.1549,Café de Upernavik
27,Kaitangata,NZ,37.99,scattered clouds,-46.2817,169.8464,Kaitangata Motor Camp
29,Tuktoyaktuk,CA,32.00,broken clouds,69.4541,-133.0374,End of The Road Ltd.
30,Sergiyevsk,RU,45.45,overcast clouds,53.9413,51.1650,Vertikal'
33,Verkhnevilyuysk,RU,24.44,overcast clouds,63.4453,120.3167,Gostinitsa Mila
38,Mataura,NZ,42.48,scattered clouds,-46.1927,168.8643,Ellie's Villa
41,Clyde River,CA,28.63,fog,70.4692,-68.5914,Qamaq Hotel
42,Muzhi,RU,31.03,overcast clouds,65.3667,64.6667,Gostinitsa
53,Iqaluit,CA,31.73,light snow,63.7506,-68.5145,Frobisher Inn


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [83]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [84]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [85]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [86]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Radisson Blu Polar Hotel, Spitsbergen</dd>\n<dt>City</dt><dd>Longyearbyen</dd>\n<dt>Country</dt><dd>SJ</dd>\n<dt>Current Weather</dt><dd>broken clouds and 40.84 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Café de Upernavik</dd>\n<dt>City</dt><dd>Upernavik</dd>\n<dt>Country</dt><dd>GL</dd>\n<dt>Current Weather</dt><dd>few clouds and 20.46 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Kaitangata Motor Camp</dd>\n<dt>City</dt><dd>Kaitangata</dd>\n<dt>Country</dt><dd>NZ</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 37.99 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>End of The Road Ltd.</dd>\n<dt>City</dt><dd>Tuktoyaktuk</dd>\n<dt>Country</dt><dd>CA</dd>\n<dt>Current Weather</dt><dd>broken clouds and 32.0 °F</dd>\n</dl>\n',
 "\n<dl>\n<dt>Hotel Name</dt><dd>Vertikal'</dd>\n<dt>City</dt><dd>Sergiyevsk</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 45.45 °F</dd>\n</dl>\n",
 '\n<dl>\n<dt>Hotel Nam

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [87]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
1,78.2186,15.6401
12,72.7868,-56.1549
27,-46.2817,169.8464
29,69.4541,-133.0374
30,53.9413,51.1650
33,63.4453,120.3167
38,-46.1927,168.8643
41,70.4692,-68.5914
42,65.3667,64.6667
53,63.7506,-68.5145


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [88]:
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))